In [ ]:
import ast
import pandas as pd

# ------------------------------------------------------------------
# Config
# ------------------------------------------------------------------
INPUT_CSV = "pde_sum_all_files.csv"
OUTPUT_CSV = "pde_sum_all_files_with_names_newlight.csv"

# ------------------------------------------------------------------
# Mapping: (ADC, local channels) -> trap/tile name
# Names built from "Light Tile Type" + "Light Tile #"
# ------------------------------------------------------------------
TRAP_NAME_MAP = [
    # ----- ADC 1 (your first block, Left) -----
    dict(adc=1, channels=[56, 57, 58, 59, 60, 61], name="ACL 4.01"),        # Pos 1 Left
    dict(adc=1, channels=[50, 51, 52, 53, 54, 55], name="LCM 49,50,51"),    # Pos 2 Left
    dict(adc=1, channels=[44, 45, 46, 47, 48, 49], name="ACL 4.21"),        # Pos 3 Left
    dict(adc=1, channels=[38, 39, 40, 41, 42, 43], name="LCM 09,08,13"),    # Pos 4 Left
    dict(adc=1, channels=[32, 33, 34, 35, 36, 37], name="ACL 4.02"),        # Pos 5 Left
    dict(adc=1, channels=[24, 25, 26, 27, 28, 29], name="LCM 19,20,21"),    # Pos 6 Left
    dict(adc=1, channels=[18, 19, 20, 21, 22, 23], name="ACL 4.07"),        # Pos 7 Left
    dict(adc=1, channels=[12, 13, 14, 15, 16, 17], name="LCM 07,14,15"),    # Pos 8 Left
    dict(adc=1, channels=[6, 7, 8, 9, 10, 11],     name="ACL 4.11"),        # Pos 9 Left
    dict(adc=1, channels=[0, 1, 2, 3, 4, 5],       name="LCM 04,05,06"),    # Pos 10 Left

    # ----- ADC 0 (Pos. 1–10 Right, first part of second block) -----
    dict(adc=0, channels=[0, 1, 2, 3, 4, 5],       name="ACL 4.04"),        # Pos 1 Right
    dict(adc=0, channels=[6, 7, 8, 9, 10, 11],     name="LCM 55,56,57"),    # Pos 2 Right
    dict(adc=0, channels=[12, 13, 14, 15, 16, 17], name="ACL 4.09"),        # Pos 3 Right
    dict(adc=0, channels=[18, 19, 20, 21, 22, 23], name="LCM 59,85,22"),    # Pos 4 Right
    dict(adc=0, channels=[24, 25, 26, 27, 28, 29], name="ACL 4.13"),        # Pos 5 Right
    dict(adc=0, channels=[32, 33, 34, 35, 36, 37], name="LCM 52,53,54"),    # Pos 6 Right
    dict(adc=0, channels=[38, 39, 40, 41, 42, 43], name="ACL 4.27"),        # Pos 7 Right
    dict(adc=0, channels=[44, 45, 46, 47, 48, 49], name="LCM 9,10,11"),     # Pos 8 Right
    dict(adc=0, channels=[50, 51, 52, 53, 54, 55], name="ACL 4.19"),        # Pos 9 Right
    dict(adc=0, channels=[56, 57, 58, 59, 60, 61], name="LCM 46,47,48"),    # Pos 10 Right

    # ----- ADC 2 (Pos. 1–10 Left, later in list) -----
    dict(adc=2, channels=[56, 57, 58, 59, 60, 61], name="ACL 4.17"),        # Pos 1 Left
    dict(adc=2, channels=[50, 51, 52, 53, 54, 55], name="LCM 43,44,45"),    # Pos 2 Left
    dict(adc=2, channels=[44, 45, 46, 47, 48, 49], name="ACL 4.23"),        # Pos 3 Left
    dict(adc=2, channels=[38, 39, 40, 41, 42, 43], name="LCM 40,41,42"),    # Pos 4 Left
    dict(adc=2, channels=[32, 33, 34, 35, 36, 37], name="ACL 4.22"),        # Pos 5 Left
    dict(adc=2, channels=[24, 25, 26, 27, 28, 29], name="LCM 34,35,36"),    # Pos 6 Left
    dict(adc=2, channels=[18, 19, 20, 21, 22, 23], name="ACL 4.08"),        # Pos 7 Left
    dict(adc=2, channels=[12, 13, 14, 15, 16, 17], name="LCM 37,38,39"),    # Pos 8 Left
    dict(adc=2, channels=[6, 7, 8, 9, 10, 11],     name="ACL 4.24"),        # Pos 9 Left
    dict(adc=2, channels=[0, 1, 2, 3, 4, 5],       name="LCM 31,32,33"),    # Pos 10 Left

    # ----- ADC 3 (Pos. 1–10 Right, last part) -----
    dict(adc=3, channels=[0, 1, 2, 3, 4, 5],       name="ACL 4.00"),        # Pos 1 Right
    dict(adc=3, channels=[6, 7, 8, 9, 10, 11],     name="LCM 28,29,30"),    # Pos 2 Right
    dict(adc=3, channels=[12, 13, 14, 15, 16, 17], name="ACL 4.06"),        # Pos 3 Right
    dict(adc=3, channels=[18, 19, 20, 21, 22, 23], name="LCM 12,24,27"),    # Pos 4 Right
    dict(adc=3, channels=[24, 25, 26, 27, 28, 29], name="ACL 4.18"),        # Pos 5 Right
    dict(adc=3, channels=[32, 33, 34, 35, 36, 37], name="LCM 17,18,23"),    # Pos 6 Right
    dict(adc=3, channels=[38, 39, 40, 41, 42, 43], name="ACL 4.15"),        # Pos 7 Right
    dict(adc=3, channels=[44, 45, 46, 47, 48, 49], name="LCM 16,25,26"),    # Pos 8 Right
    dict(adc=3, channels=[50, 51, 52, 53, 54, 55], name="ACL 4.14"),        # Pos 9 Right
    dict(adc=3, channels=[56, 57, 58, 59, 60, 61], name="LCM 01,02,03"),    # Pos 10 Right
]

# ------------------------------------------------------------------
# Helpers
# ------------------------------------------------------------------
def parse_ch_list(value):
    """
    Turn the ch_list column into a Python list of ints.
    Handles things like:
      "[0, 4]"       -> [0, 4]
      "0,4"          -> [0, 4]
      already-a-list -> itself
    """
    if isinstance(value, list):
        return value

    if pd.isna(value):
        return []

    if isinstance(value, str):
        text = value.strip()
        # case: string representation of a Python list
        if text.startswith("[") and text.endswith("]"):
            try:
                parsed = ast.literal_eval(text)
                if isinstance(parsed, (list, tuple)):
                    return [int(x) for x in parsed]
            except Exception:
                pass

        # fallback: comma-separated
        parts = [p.strip() for p in text.split(",") if p.strip()]
        out = []
        for p in parts:
            try:
                out.append(int(p))
            except ValueError:
                # ignore non-numeric garbage if any
                pass
        return out

    # anything else -> empty
    return []


def find_trap_name(adc_value, ch_list):
    """
    Find trap name by:
      - matching adc
      - checking if ANY channel in ch_list is in the mapping's channels
    If nothing matches, return "".
    """
    if ch_list is None:
        ch_list = []

    try:
        adc_int = int(adc_value)
    except Exception:
        return ""

    for entry in TRAP_NAME_MAP:
        if entry["adc"] != adc_int:
            continue
        channels = entry["channels"]
        # check intersection: any ch in row's ch_list that belongs to this trap
        if any((ch in channels) for ch in ch_list):
            return entry["name"]

    return ""


# ------------------------------------------------------------------
# Main
# ------------------------------------------------------------------
def main():
    # Read original CSV
    df = pd.read_csv(INPUT_CSV)

    # Parse ch_list into actual lists of ints
    df["ch_list_parsed"] = df["ch_list"].apply(parse_ch_list)

    # Compute trap_name for each row
    df["trap_name"] = [
        find_trap_name(row["adc"], row["ch_list_parsed"])
        for _, row in df.iterrows()
    ]

    # Drop helper column
    df = df.drop(columns=["ch_list_parsed"])

    # Write new CSV with extra column
    df.to_csv(OUTPUT_CSV, index=False)


if __name__ == "__main__":
    main()
